<a href="https://colab.research.google.com/github/SanjivMehra26/Generative-AI/blob/main/Coding_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-sml


/bin/bash: line 1: nvidia-sml: command not found


In [ ]:
!nvidia-smi

Mon Nov 11 10:53:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import pdb

def processname(name):
  pdb.set_trace() # c for comtinue, l from seeing the line , n for next line
  when = "today"
  print(name,'is using google colab',when)

processname('javier')

> <ipython-input-15-fb74a2c99318>(5)processname()
      3 def processname(name):
      4   pdb.set_trace() # c for comtinue, l from seeing the line , n for next line
----> 5   when = "today"
      6   print(name,'is using google colab',when)
      7 

ipdb> c
javier is using google colab today


In [20]:
# import the libraries

import torch, pdb
from torch.utils.data import DataLoader
from torch import nn
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from tqdm.auto import tqdm
import matplotlib.pyplot as plty

In [26]:
#visualization function
def show(tensor, ch=1, size=(28,28),num=16):
  #tensor: 128 (Batch size) * 784 (28*28)
  tensor.detach().cpu().view(-1,ch,*size) #128*1*28*28
  grid = make_grid(data[:num],nrow=4).permute(1,2,0)
  plt.imshow(grid)
  plt.show()

In [24]:
# setup of the main parameters and hyperparameters

epochs = 500
cur_step = 0
info_step = 300
mean_gen_loss = 0
mean_disc_loss = 0

z_dim = 64
lr = 0.00001
loss_func = nn.BCEWithLogitsLoss()

bs = 128
device = 'cuda'

dataloader = DataLoader(MNIST('.',download=True,transform=transforms.ToTensor()),shuffle=True,batch_size=bs)

# number if steps = 60000 / 128 =
# 60K number of image MNIST has in package

In [28]:
# declare our models

# Generator

def genBlock(inp,out):
  return nn.Sequential(
      nn.Linear(inp,out),
      nn.BatchNorm1d(out),
      nn.ReLU(inplace=True)
  )


  class Generator(nn.Module):
    def __init__(self,z_dim=64,i_dim=784,h_dim=128):
      super().__init__()
      self.gen = nn.Sequential(
          genBlock(z_dim,h_dim),
          genBlock(h_dim,h_dim*2),
          genBlock(h_dim*2,h_dim*4),
          genBlock(h_dim*4,h_dim*8),
          nn.Linear(h_dim*8,i_dim),
          nn.sigmoid(),

      )

      def forward(self,noise):
        return self.gen(noise)

def gen_noise(number,z_dim):
  return torch.randn(number,z_dim).to(device)

In [29]:
## Discriminator

def discBlock(inp,out):
  return nn.Sequential(
      nn.Linear(inp,out),
      nn.LeakyReLU(0.2)
  )

class Discriminator(nn.Module):
  def __init__(self,i_dim=784,h_dim=256):
    super().__init__()
    self.disc=nn.Sequential(
        discBlock(i_dim,h_dim*4),
        discBlock(h_dim*4,h_dim*2),
        discBlock(i_dim*2,h_dim),
        nn.Linear(h_dim,1)
    )

    def forward(self,image):
        return self.disc(image)